https://github.com/jvines/eso-query/blob/main/eso_query.py

In [1]:
import argparse
from pyvo.dal import tap
from astropy import units as u
from astroquery.mast import Catalogs
import numpy as np
from astropy.time import Time
from difflib import SequenceMatcher
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import re

ESO_TAP_OBS = 'http://archive.eso.org/tap_obs'
tap_obs = tap.TAPService(ESO_TAP_OBS)

In [2]:
def summarise_obs(t):
    if len(t) == 0:
        print('No results to show.')
        return
    # First, lets check to see if more than one object exists
    if np.unique(t['object']).shape[0] > 1:
        text = f'''
        Summary includes multiple targets: {",".join(np.unique(t['object']))}
        '''
    else:
        text = f'Summary for {t["object"][0]}'
    for instrument in np.unique(t['instrument']):
        text +='\n'
        t_ = t[t['instrument']==instrument]
        start = str(Time(np.min(t_['mjd_obs']), format='mjd').datetime)[:10]
        end = str(Time(np.max(t_['mjd_obs']), format='mjd').datetime)[:10]
        text += f'{instrument} {start} -> {end} [{len(t_):,} points]'
        for prog_id in np.unique(t_['prog_id']):
            t__ = t_[t_['prog_id']==prog_id]
            start = str(
                Time(np.min(t__['mjd_obs']), format='mjd').datetime
                )[:10]
            end = str(
                Time(np.max(t__['mjd_obs']), format='mjd').datetime
                )[:10]
            tmp = f'{prog_id} ({t__["pi_coi"][0].split("/")[0]})'
            text += f'\n\t{tmp:<40} {start} -> {end} [{len(t__):,} points]'
    return text


def text_similarity(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()


def get_tic_id(target):
    catalog = Catalogs.query_object(target,
                                    radius=1e-3, catalog='TIC')
    return f'TIC-{catalog["ID"][0]}'


def summarize_multiple_observations(table):
    table['object'] = list(map(lambda x: fix_tic(x), table['object']))

    text = ''
    for target in np.unique(table['object']):
        mask = table['object'] == target
        n_points = len(table[mask])
        instruments = ';'.join(np.unique(table[mask]['instrument']))
        start_date = (Time(np.min(table[mask]['mjd_obs']), format='mjd')
                      .datetime
                      .strftime('%Y-%m-%d'))
        end_date = (Time(np.max(table[mask]['mjd_obs']), format='mjd')
                      .datetime
                      .strftime('%Y-%m-%d'))
        pis = []
        pi_cois = np.sort(table[mask]['pi_coi'])
        for pi in pi_cois:
            pi = pi.split('/')[0]
            if pi not in pis:
                if len(list(filter(lambda x: text_similarity(pi, x) > .75, pis))):
                    continue
                pis.append(pi)
        pis = f"{';'.join(pis)}"
        target_name = table[mask]['target'][0]
        text += f'{target_name}|({instruments})|{start_date} -> {end_date} [{n_points} points]|({pis})\n'
    return text


def do_query(ra, dec, radius):
    query = f"""
    SELECT *
    FROM
    (
        SELECT
            target
            , object
            , ra
            , dec
            , pi_coi
            , prog_id
            , instrument
            , telescope
            , exp_start
            , exposure
            , mjd_obs
        --    , dp_cat
            , datalink_url
        FROM dbo.raw
        WHERE dp_cat='SCIENCE'
            AND (instrument='ESPRESSO' OR instrument='HARPS' OR instrument='FEROS')
            AND dec BETWEEN -90 AND 90
    ) AS sub
    WHERE 1=CONTAINS(
                point('', sub.ra, sub.dec),
                circle('', {ra}, {dec}, {radius}))
    """
    res = tap_obs.search(query=query)
    return res.to_table()


def build_circle_condition(ras, decs, radius):
    template = "1=CONTAINS(point('', ra, dec), circle('', {}, {}, {}))"
    condition = template.format(ras[0].value, decs[0].value, radius)
    for ra, dec in zip(ras[1:], decs[1:]):
        condition += ' OR ' + template.format(ra.value, dec.value, radius)
    return condition


def do_multiple_query(ra, dec, radius):
    condition = build_circle_condition(ra, dec, radius)
    query = f"""
    SELECT
        target
        , object
        , ra
        , dec
        , pi_coi
        , prog_id
        , instrument
        , telescope
        , exp_start
        , exposure
        , mjd_obs
    --    , dp_cat
        , datalink_url
    FROM dbo.raw
    WHERE dp_cat='SCIENCE'
        AND (instrument='ESPRESSO' OR instrument='HARPS' OR instrument='FEROS')
        AND dec BETWEEN -90 AND 90
        AND ({condition})
    """
    res = tap_obs.search(query=query)
    return res.to_table()


def fix_tic(tic_id):
    if 'TIC' in tic_id:
        pattern = r'(TIC)\s?(\d+)'
        replacement = r"\1-\2"
        return re.sub(pattern, replacement, tic_id)
    elif 'TOI' in tic_id:
        return get_tic_id_from_toi(tic_id.replace('-','').replace(' ','')[3:])
    else:
        try:
            return get_tic_id(tic_id)
        except:
            return tic_id


def get_tic_id_from_toi(toi):
    catalog = NasaExoplanetArchive.query_criteria(table='toi',
                                                  where=f'toipfx = {toi}')
    return f'TIC-{catalog["tid"][0]}'


def get_ra_dec_from_tic_id(tic_id):
    catalog = Catalogs.query_object(f'TIC {tic_id}',
                                    radius=1e-3, catalog='TIC')
    return catalog['ra'][0] * u.deg, catalog['dec'][0] * u.deg


def get_multiple_tic_ids_ra_dec(tic_ids):
    catalog = Catalogs.query_criteria(catalog='TIC', ID=tic_ids)
    return catalog['ra'] * u.deg, catalog['dec'] * u.deg

In [3]:
import sys

sys.path.insert(0, '../code/')

In [4]:
from catalog import get_tois

tois = get_tois(remove_known_planets=True)
tois.tail()

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg,GaiaDR3_exofop,GaiaDR3_vizier
6140,201175570,7041.01,TIC 201175570.01,3,4,4,3,1,4,4,...,NaN,"68,69",2024-08-28,2024-08-28,2024-09-18 00:00:00,evolved host; low SNR; long duration,357.802167,-50.869861,6.522218e+18,6.522218e+18
6141,345724317,7042.01,NaN,2,4,2,2,1,4,4,...,0.184685,"25,52,79",2024-08-28,2024-08-28,2024-09-08 12:03:03,slight depth aperture correlation; weak BLS; l...,249.910167,20.743061,4.564173e+18,4.564173e+18
6142,57299130,7043.01,NaN,3,4,3,3,1,4,4,...,0.109921,"24,25,51,52,78,79",2024-08-28,2024-08-28,2024-09-08 12:03:02,some events on momentum dumps which cause appa...,251.806167,37.885958,1.352263e+18,1.352263e+18
6143,263723967,7044.01,NaN,3,4,3,3,1,4,4,...,0.136806,"25,52,79",2024-08-28,2024-08-28,2024-09-08 12:03:02,maximum period between events in each year is ...,250.414750,15.200531,4.462612e+18,4.462612e+18
6144,21720215,7045.01,TIC 21720215.01,3,4,3,3,1,4,4,...,0.192469,"25,79",2024-08-28,2024-08-28,2024-09-08 12:03:00,two stars in pixel; somewhat V-shaped,256.054042,31.565369,1.310410e+18,1.310410e+18


# multiple queries at once but hard to distinguish target name

In [5]:
tic_list = tois['TIC ID'].T.values[:5]
tic_list

array([278683844, 278683844, 207081058, 231702397,  29831208])

In [6]:
get_multiple_tic_ids_ra_dec(tic_list)

(<Quantity [ 99.23800267, 332.94708256,  66.58254045, 331.86723032] deg>,
 <Quantity [-58.01526792, -58.94506972, -67.80649444, -41.81546905] deg>)

In [32]:
radius = (10 * u.arcsec).to(u.deg)
outdir = '.'
ras, decs = get_multiple_tic_ids_ra_dec(tic_list)
results = do_multiple_query(ras, decs, radius.value)
print(summarize_multiple_observations(results))

HATS582-005|(FEROS)|2012-04-21 -> 2012-06-08 [13 points]|(MOHLER, M)
J054833|(FEROS)|2011-12-17 -> 2011-12-17 [1 points]|(FAEDI)
J211947|(FEROS)|2011-12-18 -> 2011-12-18 [3 points]|(FAEDI)
SW0548-6359|(HARPS)|2012-10-12 -> 2012-10-17 [15 points]|(TRIAUD)
WASP-95|(FEROS)|2014-11-21 -> 2014-11-21 [1 points]|(MANCINI, L)
WASP-62|(FEROS;HARPS)|2014-10-11 -> 2017-01-26 [5 points]|(LILLO BOX;MANCINI, L)
WASP46|(HARPS)|2018-11-05 -> 2018-11-06 [15 points]|(ESPOSITO)
TOI104|(FEROS;HARPS)|2015-07-25 -> 2024-07-19 [2 points]|(HUBER ;MANCINI, L)



## loop each target

In [15]:
import pandas as pd

candidates = pd.read_csv('../../tois_yuanzhe/tois_yuanzhe.txt', names=['TOI']).squeeze()
candidates

0     7038.01
1     6963.01
2     6938.01
3     6715.01
4     6705.01
       ...   
65     612.01
66     588.01
67     581.01
68     580.01
69     520.01
Name: TOI, Length: 70, dtype: float64

In [24]:
idx = tois.TOI.isin(candidates)
tois[idx].shape

(65, 66)

In [26]:
results = {}
texts = {}

In [29]:
from tqdm import tqdm

radius = (10 * u.arcsec).to(u.deg)

for i,row in tqdm(tois[idx].iterrows()):
    toi = f"TOI-{row['TOI']}"
    # print(row['Comments'])
    if results.get(toi) is None:
        ra, dec = get_ra_dec_from_tic_id(row['TIC ID'])
        result = do_query(ra.value, dec.value, radius.value)
        text = summarise_obs(result)
        results[toi] = result
        texts[toi] = text
        print(text)

2it [00:08,  4.31s/it]

No results to show.
None


3it [00:15,  5.50s/it]

No results to show.
None


4it [00:23,  6.17s/it]

...

No results to show.
None


64it [07:32,  6.98s/it]

No results to show.
None


65it [07:40,  7.08s/it]

No results to show.
None


In [38]:
[k for k,v in results.items() if v]

['TOI-612.01', 'TOI-1881.01', 'TOI-1882.01', 'TOI-6705.01', 'TOI-6715.01']

In [54]:
[(k,set(v['instrument'])) for k,v in results.items() if v]

[('TOI-612.01', {'HARPS'}),
 ('TOI-1881.01', {'HARPS'}),
 ('TOI-1882.01', {'HARPS'}),
 ('TOI-6705.01', {'HARPS'}),
 ('TOI-6715.01', {'HARPS'})]

Download data of each TOI above from: https://archive.eso.org/wdb/wdb/adp/phase3_spectral/form?collection_name=HARPS

In [41]:
tois.query("TOI==6715.01")[['ra_deg','dec_deg']]

,ra_deg,dec_deg
5820,159.157958,-64.798231


PI:Dan Huber; not yet public until 2025 Feb

In [55]:
tois.query("TOI==6705.01")[['ra_deg','dec_deg']]

,ra_deg,dec_deg
5811,220.003917,-52.91005


In [56]:
tois.query("TOI==1882.01")[['ra_deg','dec_deg']]

,ra_deg,dec_deg
1374,212.839208,-75.833203
